## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,-34.28,80,0,6.85,clear sky
1,1,Kruisfontein,ZA,-34.0033,24.7314,60.01,90,0,3.51,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,86.00,79,75,13.80,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.80,62,0,26.46,clear sky
4,4,Mount Gambier,AU,-37.8333,140.7667,64.40,59,75,8.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
len(preferred_cities_df)

154

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                154
City                   154
Country                153
Lat                    154
Lng                    154
Max Temp               154
Humidity               154
Cloudiness             154
Wind Speed             154
Current Description    154
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Mataura,NZ,71.98,overcast clouds,-46.1927,168.8643,
12,Daniel Flores,CR,70.14,overcast clouds,9.3355,-83.6694,
18,Vila Velha,BR,78.80,few clouds,-20.3297,-40.2925,
24,Busselton,AU,79.00,clear sky,-33.6500,115.3333,
27,Bryan,US,71.60,clear sky,30.6744,-96.3700,
32,Rikitea,PF,79.50,clear sky,-23.1203,-134.9692,
41,San Patricio,US,73.40,few clouds,28.0170,-97.5169,
46,Cabo San Lucas,MX,72.00,scattered clouds,22.8909,-109.9124,
56,Mabaruma,GY,74.64,clear sky,8.2000,-59.7833,
58,Soyo,AO,79.68,light rain,-6.1349,12.3689,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except(IndexError):
        #hotel_df.loc[index, "Hotel Name"] = ""
        print("Hotel not found... skipping.")
print("Finished")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Finished


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != "")]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Mataura,NZ,71.98,overcast clouds,-46.1927,168.8643,Ellie's Villa
12,Daniel Flores,CR,70.14,overcast clouds,9.3355,-83.6694,Hotel San Isidro
18,Vila Velha,BR,78.80,few clouds,-20.3297,-40.2925,Hotel Vitória Palace
24,Busselton,AU,79.00,clear sky,-33.6500,115.3333,Observatory Guest House
27,Bryan,US,71.60,clear sky,30.6744,-96.3700,Best Western Premier Bryan College Station
...,...,...,...,...,...,...,...
662,Axim,GH,78.93,broken clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
674,Grajau,BR,72.88,overcast clouds,-5.8194,-46.1386,Hotel Confiança
678,Port-Gentil,GA,77.68,overcast clouds,-0.7193,8.7815,Fengshui Residence
680,Bolitoc,PH,74.62,overcast clouds,15.7395,119.8769,Seasun Beach Resort & Hotel


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))